# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [13]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [14]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [15]:
from ovs.indifference_zone import KN

In [16]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [17]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[ 53  28  29  30  31  54 116 317 363 364]
total reps	1385
means		[1.13 2.35 3.11 3.99 5.02 5.91 6.93 7.99 9.08 9.95]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [18]:
from ovs.fixed_budget import OCBA, OCBAM

In [19]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='min')

In [20]:
print(ocba)

OCBA(n_designs=10, budget=500, delta=5, n_0=5, obj=min)


call the `solve()` method to run the optimisation

In [21]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	0
allocations:	[218 200  41   9   6   6   5   5   5   5]
total reps:	500
means:		[ 1.01  1.91  2.99  4.33  4.85  5.73  7.2   7.24  8.99 10.62]
vars:		[1.01 0.91 0.87 0.56 0.4  0.83 0.48 0.64 0.59 1.56]


## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [22]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocbam = OCBAM(model=environment, 
            n_designs=len(designs), 
            budget=2000, 
            delta=10, 
            n_0=5, 
            m=3,
            obj='max')

In [23]:
print(ocbam)

OCBA(n_designs=100, m=3, budget=2000, delta=10, n_0=5, obj=max)


In [24]:
results = ocbam.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocbam._allocations))
print('total reps:\t{}'.format(ocbam._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocbam._means))
print('vars:\t\t{0}'.format(ocbam._vars))
print('SEs:\t\t{0}'.format(ocbam._ses))

best designs:	[97 98 99]
allocations:	[  5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5
   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5
   5   5   6   5   5   5   5   5   5   5   5   5   5   5   6   6   5   6
   5   5   6   5   5   5   5   6   5   5   9   9   6   6   9   5   5   8
   7   7   5  12   8   8   5  12  11  11  15  15  18  17  23  18  20  15
  27  31  44  43  64  82 115 426 426 132]
total reps:	2000
means:		[ 1.1   1.49  2.75  4.65  5.95  6.36  6.85  8.26  8.89 10.17 10.53 12.07
 13.5  13.47 14.8  15.93 17.25 17.59 19.2  19.51 21.53 23.19 23.5  23.87
 24.51 25.87 26.97 28.41 29.05 30.47 31.14 32.6  32.41 34.21 34.94 35.36
 37.27 37.55 39.17 39.76 41.   42.07 43.23 44.37 43.94 44.69 47.06 48.57
 49.38 50.85 50.53 51.81 52.68 53.62 54.87 56.5  56.84 58.53 59.5  59.74
 60.76 62.32 63.46 64.26 64.67 66.11 66.6  68.34 69.49 70.06 70.65 71.48
 72.82 74.93 74.57 75.88 77.6  77.91 79.49 80.07 81.36 82.04 82.82 84.16
 84.88 86.19 87.48 

In [25]:
np.argsort(ocba._means)[-3:]

array([7, 8, 9])

# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results